In [1]:
from transformers.models.clip.modeling_clip import _expand_mask, CLIPTextTransformer, CLIPPreTrainedModel, CLIPModel
import torchvision.transforms as T

/home/tyk/anaconda3/envs/animatediff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset

dataset_name = "huggan/smithsonian_butterflies_subset"
dataset = load_dataset(dataset_name, split="train",
                       cache_dir='/home/tyk/project2/SIDM/downloads')

Found cached dataset parquet (/home/tyk/project2/SIDM/downloads/huggan___parquet/huggan--smithsonian_butterflies_subset-7665b1021a37404c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [3]:
preprocess = T.Compose([T.Resize((224, 224)),
                        T.ToTensor()])

def transform(examples):
    images = [preprocess(image.convert("RGB")) for image in examples['image']]
    return {"images" : images}

dataset.set_transform(transform)

In [4]:
model = CLIPModel.from_pretrained('openai/clip-vit-large-patch14', 
                          cache_dir='/home/tyk/project2/SIDM/downloads')

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [6]:
vision_model = model.vision_model
visual_proejction = model.visual_projection
visual_processor = T.Normalize(
            (0.48145466, 0.4578275, 0.40821073),
            (0.26862954, 0.26130258, 0.27577711),
        )
        

In [7]:
import torch
train_loader = torch.utils.data.DataLoader(dataset, batch_size=4)

In [8]:
next(iter(train_loader))['images']

tensor([[[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

         [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1

In [10]:
pixel_values = visual_processor(next(iter(train_loader))['images'])
vision_model(pixel_values).keys()
# embs = vision_model(pixel_values)[1]

: 

: 

In [26]:
vision_model.config

CLIPVisionConfig {
  "attention_dropout": 0.0,
  "dropout": 0.0,
  "hidden_act": "quick_gelu",
  "hidden_size": 1024,
  "image_size": 224,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "model_type": "clip_vision_model",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 14,
  "projection_dim": 768,
  "transformers_version": "4.31.0"
}

In [23]:
vision_model()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (672x224 and 1024x768)

In [ ]:
class FastComposerCLIPImageEncoder(CLIPPreTrainedModel):
    @staticmethod
    def from_pretrained(global_model_name_or_path):
        model = CLIPModel.from_pretrained(global_model_name_or_path,
                                          cache_dir='/home/tyk/project2/SIDM/downloads')
        vision_model = model.vision_model
        visual_projection = model.visual_projection
        visual_processor = T.Normalize(
            (0.48145466, 0.4578275, 0.40821073),
            (0.26862954, 0.26130258, 0.27577711),
        )
        return FastComposerCLIPImageEncoder(vision_model, 
                                            visual_projection,
                                            visual_processor)
    
    def __init__(self, 
                 vision_model, 
                 visual_projection,
                 visual_processor):
        super().__init__(vision_model.config)
        self.vision_model = vision_model
        self.visual_projection = visual_projection
        self.visual_processor = visual_processor

        self.image_size = vision_model.config.image_size

    def forward(self, object_pixel_values):
        b, num_ob